# Greenhouse gas emissions of pedestrian car traffic in Helsinki

In [ ]:
import math
import re
import pandas as pd
import numpy as np


def read_muni_data():
    url = 'http://lipasto.vtt.fi/liisa/kunnat2017.xlsx'
    df = pd.read_excel(url, skiprows=9, header=0)
    # Drop columns with no values
    df.dropna(axis=1, how='all', inplace=True)
    # Drop rows with any non-values
    df.dropna(axis=0, how='any', inplace=True)
    # Rename columns
    columns_left = list(df.columns)
    column_map = {
        columns_left.pop(0): "kunta",
        columns_left.pop(0): "tyyppi",
    }
    while len(columns_left):
        column_name = columns_left.pop(0)
        # Column names are like "kulutus [t]". Split the strings into
        # quantity, unit pairs.
        quantity, unit = re.search(r'([\w\.\s]+)\[(\w+)\]', column_name).groups()
        quantity = quantity.strip()
        if quantity == 'CO2 ekv.':
            quantity = 'CO2e'
        column_map[column_name] = quantity
        # print("%s [%s]" % (quantity, unit))

    df.rename(index=str, columns=column_map, inplace=True)
    df.set_index(['kunta', 'tyyppi'], inplace=True)

    # Drop summary rows
    for level_name in list(df.index.levels[1]):
        if 'yhteensä' in level_name.lower():
            df.drop(index=level_name, level=1, inplace=True)

    # 'suorite' is in Mkm, convert to km
    df['suorite'] *= 1000000

    return df

muni = read_muni_data()
display(muni.xs('Helsinki'))

In [ ]:
def get_km_share_per_engine_type():
    # Returns: percentage of kms per engine type (%)
    url = "http://lipasto.vtt.fi/aliisa/suoritejakaumat.xlsx"
    df = pd.read_excel(url, skiprows=4, header=0, index_col=0, usecols="A:I")

    # Make sure we have parsed the .xlsx correctly
    assert df.index.name == 2017

    # Remove useless rows from dataset
    df.dropna(axis=0, inplace=True, how='all')
    df.drop(['Yhteensä', 2017], inplace=True)
    # Make sure the keys we're interested in are in the data
    assert 'HA bensiini' in df.index
    assert 'HA diesel' in df.index
    return df


df = get_km_share_per_engine_type()
display(df.style.format("{:.2%}"))
share_per_eng = df['Yhteensä']

In [ ]:
def get_emission_factors_by_engine_type():
    # Returns: avg. CO2e per engine type (g/km)

    # sources:
    #    petrol: "http://lipasto.vtt.fi/yksikkopaastot/henkiloliikenne/tieliikenne/henkiloautot/habens.xlsx"
    #    diesel: "http://lipasto.vtt.fi/yksikkopaastot/henkiloliikenne/tieliikenne/henkiloautot/hadies.xlsx"
    #       BEV: "http://lipasto.vtt.fi/yksikkopaastot/henkiloliikenne/tieliikenne/henkiloautot/hasahko.xlsx"

    # GHG emissions from electric energy production is based on HSY 2017 GHG data.

    VALUES = {
        "HA diesel": {  # diesel passenger cars
            "HA tiet": 121.077115422271,
            "HA kadut": 194.769503438253,
        },
        "HA bensiini": {  # petrol passenger cars
            "HA tiet": 140.485273587607,
            "HA kadut": 209.541088794964
        },
        "HA sähkö BEV": {  # battery-powered electric cars
            "HA tiet": 0.20 * 103.6,   # (kWh/km) * (g (CO2e)/kWh) = g/km
            "HA kadut": 0.17 * 103.6,
        }
    }
    index, series = zip(*[((i, j), VALUES[i][j]) for i in VALUES for j in VALUES[i]])
    index = pd.MultiIndex.from_tuples(index, names=['power_source', 'road_type'])
    return pd.DataFrame(list(series), index=index, columns=['emission_factor'])

co2e_per_eng = get_emission_factors_by_engine_type()
display(co2e_per_eng)

In [ ]:
def calculate_co2e_per_engine_type(share_per_engine_type):
    # df1 = yearly passenger car kms in Helsinki
    df1 = muni.xs('Helsinki')['suorite'].filter(like='HA')
    # df2 = ratios of passenger cars on the roads by engine type
    df2 = share_per_engine_type.filter(like='HA')

    kms_per_eng = pd.DataFrame(np.outer(df1, df2), index=df1.index, columns=df2.index)

    out = kms_per_eng * engine_emission_factors.unstack(level=0)['emission_factor']
    out /= 1000000000  # convert to kt (CO2e)
    return out


muni = read_muni_data()
engine_emission_factors = get_emission_factors_by_engine_type()
share_per_eng = get_km_share_per_engine_type()['Yhteensä']


def bass_diffuse(t, p, q):
    e1 = math.e ** (-(p + q) * t)
    res = ((p + q) ** 2) / p
    res *= e1 / ((1 + q / p * e1) ** 2)
    return res


def test_diffuse(x0, m, p, q):
    data = []
    for t in range(0, 20):
        x0 *= 1 + bass_diffuse(t, p, q) * m
        data.append((2018 + t, x0))
    zdata = list(zip(*data))
    s = pd.Series(zdata[1], index=zdata[0])
    print(s)
    s.plot.line()


def estimate_co2e_from_electric_car_share_increase(year):
    # Assume BEV share is increasing according to the Bass diffusion model
    # and that increase in share comes equally out of petrol and diesel engines.
    df = share_per_eng.copy()
    bev_share = df['HA sähkö BEV']
    for t in range(year - 2018):
        bev_share *= 1 + bass_diffuse(t, 0.05, 0.38) * 9

    share_change = bev_share - df['HA sähkö BEV']
    m = share_change / (df['HA diesel'] + df['HA bensiini'])
    df['HA diesel'] -= m * df['HA diesel']
    df['HA bensiini'] -= m * df['HA bensiini']
    df['HA sähkö BEV'] = bev_share

    df.name = year
    df.index.name = None

    return df


yearly_dfs = []
for year in range(2019, 2035):
    share = estimate_co2e_from_electric_car_share_increase(year)
    co2e = calculate_co2e_per_engine_type(share)
    out = share.filter(items=['HA diesel', 'HA bensiini', 'HA sähkö BEV'])
    out['CO2e'] = co2e.sum().sum()
    yearly_dfs.append(out)

out = pd.DataFrame(yearly_dfs)
out.index.name = 'year'
display(out.style.format("{:.2%}").format({'CO2e': '{:.0f}'}))


In [ ]:
import plotly
import plotly.graph_objs as go
import cufflinks as cf

plotly.offline.init_notebook_mode(connected=True)
cf.set_config_file(offline=True)
fig1 = out.drop('CO2e', axis=1).iplot(kind='line', asFigure=True)
fig2 = go.Bar(x=out.index, y=out['CO2e'], yaxis='y2', name='CO₂-ekv.', opacity=0.2)

layout = go.Layout(
    xaxis=dict(title='year'),
    yaxis1=dict(overlaying='y2', tickformat=',.0%', rangemode='nonnegative'),
    yaxis2=dict(side='right', tickformat=',.0f', hoverformat='.0f', rangemode='nonnegative', title='kt'))
fig = go.Figure(data=(fig2,) + fig1.data, layout=layout)

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)